1,prepare data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras.regularizers import L1L2
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import numpy
import time


Using TensorFlow backend.


In [2]:
def preparedata_steps(steps):
    df=pd.read_csv('SP500_new.csv', engine='python', skipfooter=1)
    #data=df.drop(['Date'], axis = 1)
    data=df.drop(['Date', 'Adj Close','Volume'], axis = 1)
    # normalize the dataset
    scaler_dataset = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler_dataset.fit_transform(data)
    
    target=df['Adj Close']
    # normalize the dataset
    scaler_target = MinMaxScaler(feature_range=(0, 1))
    target = scaler_target.fit_transform(target)
    
    result = []
    for index in range(len(dataset) - steps): # maxmimum date = lastest date - sequence length
            result.append(dataset[index: index + steps]) # index : index + steps
            
    result = np.array(result)
    
    features=result[:]
    target=target[steps:]
    
#     trainX=features[:-500]
#     testX=features[-500:]
#     trainY=target[:-500]
#     testY=target[-500:]
    return features,target,scaler_dataset,scaler_target

In [3]:
# fit an LSTM network to training data
def fit_lstm(trainX, trainY, nb_epoch, neurons):
    model = Sequential()
    model.add(LSTM(neurons, batch_input_shape=(1, trainX.shape[1], trainX.shape[2]), stateful=True,bias_regularizer=L1L2(l1=0.01, l2=0.0)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='sgd')
    for i in range(nb_epoch):
        model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=0, shuffle=False)
        model.reset_states()
    return model

In [4]:
# Update LSTM model
def update_model(model, trainX,trainY, updates):
    for i in range(updates):
        model.fit(trainX,trainY, nb_epoch=1, batch_size=1, verbose=0, shuffle=False)
        model.reset_states()

In [5]:
start = time.time()

Test_true=[]
Test_pred=[]

features,target,scaler_dataset,scaler_target=preparedata_steps(15)

# fit the base model
trainX_base=features[:-500]
trainY_base=target[:-500]
model = fit_lstm(trainX_base,trainY_base,100,7)

# initialize the invent,start with 100K Au$ at day 500 whose adjclose is $2077.419922
invests=[]
invests.append(100)


for i in range(371):
    if i>0:
        update_model(model, trainX,trainY,1)
    
    testX=features[-(500-i):-(499-i)]
    # make predictions
    Prediction = model.predict(testX)
    
    # invert predictions
    Prediction =  scaler_target.inverse_transform(Prediction)
    # store forecast
    Test_pred.append(Prediction[0][0])
    
    # invert true value
    true = scaler_target.inverse_transform(target[-(500-i):-(499-i)])
    # store true value
    Test_true.append(true[0])
    
    # trade based on the predition
    if Prediction>scaler_target.inverse_transform(target[-(501-i):-(500-i)]):
        invests.append(invests[i]*(true/scaler_target.inverse_transform(target[-(501-i):-(500-i)])))
    else:
        invests.append(invests[i]*(scaler_target.inverse_transform(target[-(501-i):-(500-i)])/true))
    
     
    
    #expand the training set
    trainX=features[:-(499-i)]
    trainY=target[:-(499-i)]
    
    if i>0 and i%10==0:
        print('from day_1 to day_'+str(i))
        print('test_RMSE:'+str(math.sqrt(mean_squared_error(Test_true, Test_pred))))
        print('test_R2Score:'+str(r2_score(Test_true, Test_pred)))
        
        #transfer to classification probelm
        print('accuracy plot') 
        test_true=Test_true[:]
        test_pred=Test_pred[:]

        test_true_clssfi=[]
        for  i in range(1,len(test_true)):
            if test_true[i]>test_true[i-1]:
                test_true_clssfi.append(1)
            else:
                test_true_clssfi.append(0)

        test_pred_clssfi=[]
        for  i in range(1,len(test_pred)):
            if test_pred[i]>test_true[i-1]:
                test_pred_clssfi.append(1)
            else:
                test_pred_clssfi.append(0)

        accuracy=[]
        for i in range(1,len(test_pred_clssfi)+1):
            accuracy.append(accuracy_score(test_true_clssfi[0:i],test_pred_clssfi[0:i]))
        plt.plot(accuracy)
        plt.show()

        print('adjclose plot')  
        fig, ax = plt.subplots(figsize=(100, 50))
        plt.plot(Test_true,color='red')
        plt.plot(Test_pred,color='blue')
        plt.show()
        
        print('investment plot')  
        plt.plot(invests)
        plt.show()


    
    
    


C:\Program Files\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


KeyboardInterrupt: 

In [9]:
results_sgd_invests = pd.DataFrame({
        "invests": invests,
    })
results_sgd_invests.to_csv('results_sgd_invests.csv', index=False)

In [10]:
results_sgd_Test_true = pd.DataFrame({
        "Test_true": Test_true
    })
results_sgd_Test_true.to_csv('results_sgd_Test_true.csv', index=False)

In [11]:
results_sgd_Test_pred = pd.DataFrame({
        "Test_pred": Test_pred
    })
results_sgd_Test_pred.to_csv('results_sgd_Test_pred.csv', index=False)

In [12]:
results_sgd_accuracy = pd.DataFrame({
        "accuracy": accuracy
    })
results_sgd_accuracy.to_csv('results_sgd_accuracy.csv', index=False)